In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
import numpy as np
import scipy.signal

# Transformada de Fourier para señales digitales

## Transformada de Fourier Discreta (DFT)


En este curso nos enfocaremos en el procesamiento de señales usando software

Por ende asumiremos que existe un proceso que sensa y convierte la señal analógica continua en una señal digital, como muestre el siguiente diagrama

<img src="../images/signal-sampling1.png" width="500">

- La señal análogica $s(t)$ se muestrea cada $T_s$ durante un tiempo $T$
- El tiempo asociada a $s_n$ es $t_n = n \cdot T_s$ [s]
- El inverso del periódo de muestreo se denomina **frecuencia de muestreo**: $F_s = \frac{1}{T_s}$ [Hz]
- La cantidad de muestras del arreglo $\{s_n\}$ es equivalente a $N = \lfloor T F_s \rfloor $


Más formalmente diremos que existe un sistema muestreador con frecuencia de muestreo $F_s$ [Hz] tal que

$$
s(t) = \sum_{n=0}^{N-1} s[n] \delta(t - n/F_s),
$$

La transformada de Fourier de esta expresión es

$$
\begin{align}
S(\omega) &= \int s(t) e^{-j\omega t} dt \nonumber \\
&= \int \sum_{n=0}^{N-1} s[n] \delta(t - n/F_s) e^{-j\omega t} dt \nonumber \\
&=  \sum_{n=0}^{N-1} s[n] \int \delta(t - n/F_s) e^{-j\omega t} dt \nonumber \\
&=  \sum_{n=0}^{N-1} s[n] e^{-j\omega n/F_s} \nonumber 
\end{align}
$$

Finalmente definiendo $\omega = 2 \pi f = 2 \pi k \Delta f$ donde $\Delta f = \frac{1}{T} = \frac{F_s}{N}$ y reemplazando obtenemos

$$
S[k] =  \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} k n},
$$

donde $k = [0, 1, \ldots N-1]$, que se conoce como la **transformada de Fourier discreta** o *discrete Fourier Transform (DFT)

En esta expresión:

- El índice $n$ representa la discretización en el tiempo
- El índice $k$ representa la discretización en frecuencia

### Propiedades de la DFT

La DFT comparte las propiedades de la FT que vimos en la lección anterior

Sin embargo, a diferencia de la FT para señales continuas, la DFT es periódica y su período $N$, como muestra la siguiente expresión

$$
\begin{align}
S[k+N] &= \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} (k+N) n} \nonumber \\
&=   \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} k n} e^{-j 2 \pi n} \nonumber \\
&= \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} k n} = S[k]\nonumber 
\end{align}
$$

### Rango frecuencial de la DFT y frecuencia de Nyquist

Considerando la periodicidad de la DFT, la correspondencia entre el índice entero $k$ y la frecuencia en Hertz es

$$
\begin{matrix}
k &  : & 0, & 1, & 2, & \ldots & N/2 -1, & N/2, & N/2+1, & \ldots & N-2, & N-1 \\
f = k \frac{F_s}{N} & : & 0, & \frac{F_s}{N}, & \frac{2 F_s}{N}, & \ldots & \frac{F_s}{2} - \frac{F_s}{N}, & \frac{F_s}{2} = -\frac{F_s}{2}, &  \frac{F_s}{2} + \frac{F_s}{N} = - \frac{F_s}{2}  + \frac{F_s}{N}, & \ldots & -\frac{2 F_s}{N},  & -\frac{F_s}{N}
\end{matrix}
$$

donde $\frac{F_s}{2}$ se conoce como **frecuencia de Nyquist**, y es la frecuencia más alta a la que se puede calcular la DFT. Hablaremos más sobre esto en la próximo lección

### Interpretación de la DFT es  un producto matricial

Sea $\{s_n\}_{n=0,\ldots,N-1}$ y definiendo 

$$
W_N = e^{-j \frac{2\pi}{N}} = \cos \left(\frac{2\pi}{N}\right) - j \sin \left(\frac{2\pi}{N}\right)
$$

podemos expresar la transformada de Fourier discreta como

$$
S[k] =  \sum_{n=0}^{N-1} s[n] W_N^{kn}, \quad k = [0, 1, \ldots N-1],
$$

que también puede ser expresado matricialmente como

$$
\begin{align}
\begin{pmatrix} 
S[0] \\
S[1] \\
S[2] \\
\vdots \\
S[N-1] \\
\end{pmatrix} &=
\begin{pmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & W_N & W_N^2 & \cdots & W_N^{N-1} \\
1 & W_N^2 & W_N^4 & \cdots & W_N^{N-2} \\
\vdots & \dots & \dots & \ddots &  \vdots \\
1 & W_N^{N-1} & W_N^{N-2} & \cdots & W_N \\
\end{pmatrix} 
\begin{pmatrix} 
s[0] \\
s[1] \\
s[2] \\
\vdots \\
s[N-1] \\
\end{pmatrix} \nonumber  \\
S &= \Omega s,
\end{align}
$$


Notemos que:

- Por definición $W_N^{kn} = \left(e^{-j \frac{2\pi}{N}}\right)^{kn} = e^{-j \frac{2\pi}{N}kn}$
- Por periodicidad $W_N^{2(N-1)} = W_N^{2(N-1) - N}  = W_N^{N-2}$
- También se tiene simetría hermítica: $W_N^{k(-n)} = W_N^{-kn} = (W_N^{kn})^*$
- $\Omega$ es una matriz cuadrada y simétrica 
- El cálculo de la DFT tiene complejidad cuadrática: $N^2$ multiplicaciones y $N$ sumas


### DFT inversa

Sea $S=\{S[0], S[1], S[2], S[3]\}$, la DFT de una señal $s=\{s[0], s[1], s[2], s[3]\}$. 

Como vimos anteriormente podemos escribir la relación entre $S$ y $s$ como el siguiente sistema de ecuaciones

$$
S= 
\begin{pmatrix} 
S[0] \\
S[1] \\
S[2] \\
S[3] 
\end{pmatrix} =
\begin{pmatrix}
1 & 1  & 1 & 1\\
1 & W_4  & W_4^2 & W_4^3 \\
1 & W_4^2  & W_4^4 & W_4^2 \\
1 & W_4^3  & W_4^2 & W_4 \\
\end{pmatrix} 
\begin{pmatrix} 
s[0] \\
s[1] \\
s[2] \\
s[3] 
\end{pmatrix} = \begin{pmatrix}
1 & 1  & 1 & 1\\
1 & -j  & -1 & j \\
1 & -1  & 1 & -1 \\
1 & j  & -1 & -j \\
\end{pmatrix} 
\begin{pmatrix} 
s[0] \\
s[1] \\
s[2] \\
s[3] 
\end{pmatrix} = \Omega s
$$

Si conocemos $S$ y queremos encontrar $s$, sólo necesitamos invertir la matriz $\Omega$

Por ejemplo

In [ ]:
np.set_printoptions(precision=4, suppress=True)

N = 4
WN = np.exp(-1j*2.0*np.pi/N)
index = np.array(range(N))[:, np.newaxis]
Omega = WN**(index*index.T)

print(Omega)

np.linalg.inv(Omega) # Estación función calcula el inverso de una matriz cuadrada

Como veremos a continuación la matriz $\Omega$ de la DFT es una matriz bastante especial. 

El inverso de $\Omega$ es equivalente a su complejo conjugado dividido N. El complejo conjugado de una matriz es una matriz donde todos los elementos complejos se reemplazan por su conjugado 

$$
(a + j b)^{*} = a - j b
$$

En este caso particular ($N=4$) tenemos que el inverso de la matriz es 

$$
\Omega^{-1} = \frac{1}{4}
\begin{pmatrix}
1 & 1 & 1 &  1 \\
1 & j & -1 &  -j \\
1 & -1 & 1 &  -1 \\
1 & -j & -1  & j \\
\end{pmatrix} = \frac{1}{4} \Omega^*
$$

Notar que esta propiedad se cumple para todo $N$. Esto es muy útil pues el costo de obtener el complejo conjugado es mucho menor que el de invertir una matriz cuadrada arbitraria

Resumiendo, podemos recuperar $s$ a partir de $S$ usando

$$
s = \frac{1}{N} \Omega^* S
$$

o lo que es equivalente

$$
s[n] = \frac{1}{N} \sum_{k=0}^{N-1} S[k] W_N^{-kn}, \quad n = [0, 1, \ldots N-1]
$$

que se conoce como la DFT inversa y cuya únicas diferencias con la DFT son el factor $\frac{1}{N}$ y el signo del exponente

## Transformada Rápida de Fourier (FFT)

Como vimos antes la computación de la DFT tiene complejidad $\mathcal{O}(N^2)$, lo cual es muy costoso de aplicar en la práctica

Existe una aproximación exacta de la DFT con complejidad $\mathcal{O}(N\log N)$: la **Fast Fourier Transform** (FFT). 

En esta lección veremos el algoritmo de Cooley-Tukey, un algoritmo FFT, que se basa en expresiones recursiva que explotan las simetrías que vimos en la matriz $\Omega$

La DFT se separa en dos "medias" DFT, donde una mitad se enfoca en los índices pares (even) y la otra en los impares (odd) como muestra la siguiente ecuación

$$
\begin{align}
S[k] &=  \sum_{n=0}^{N-1} s[n] W_N^{kn} \nonumber \\
&= \sum_{n=0}^{N/2-1} s[2n] W_N^{k 2n} + \sum_{n=0}^{N/2-1} s[2n+1] W_N^{k(2n+1)} \nonumber \\
&= \sum_{n=0}^{N/2-1} s[2n] W_{N/2}^{kn} + W_N^{k} \sum_{n=0}^{N/2-1} s[2n+1] W_{N/2}^{kn} \nonumber \\
&= S_E[k] + W_N^{k} S_O[k] ~~ \forall k \in [0,N/2]  \nonumber 
\end{align} 
$$

Luego por periodicidad de la DFT tenemos que

$$
\begin{align}
S_E[k + N/2] &=  \sum_{n=0}^{N/2-1} s[2n] W_{N/2}^{(k+N/2)n} \nonumber \\
&=  \sum_{n=0}^{N/2-1} s[2n] W_{N/2}^{kn} \exp \left(-j2\pi n \right) = S_E[k], \nonumber
\end{align}
$$

e igualmente

$$
S_O[k + N/2] = S_O[k],
$$

juntando ambos tenemos que

$$
\begin{align}
S[k + N/2] &=  S_E[k + N/2] + W_{N}^{(k+N/2)} S_O[k + N/2] \nonumber  \\
&=  S_E[k] + W_{N}^{k} \exp \left(-j\pi\right) S_O[k] \nonumber \\
&=  S_E[k] - W_{N}^{k} S_O[k] \nonumber 
\end{align}
$$

es decir que

$$
\begin{align}
S[k] &=  S_E[k] + W_{N}^{k} S_O[k] \nonumber \\
S[k + N/2] &=  S_E[k] - W_{N}^{k} S_O[k] \quad \forall k \in [0,N/2]  \nonumber 
\end{align}
$$

La DFT de $k$ y $k+N/2$ difieren sólo en un signo. Cada ves que dividimos el intervalo podemos reducir los cómputos a la mitad. Podemos repetir la división $\log N$ veces por lo que el costo de la FFT es $N \log N$

La siguiente esquema muestra la FFT para una señal $x$ de largo $N=16$, en $\log_2 16=4$  pasos se calculan todos los componentes de su espectro $X$

<img src="../images/fft-16samples.png">

Imagen tomada de: [http://www.themobilestudio.net/the-fourier-transform-part-14](http://www.themobilestudio.net/the-fourier-transform-part-14)

La siguiente tabla compara la cantidad de multiplicaciones entre la DFT y la FFT

| N | DFT | FFT | FFT/DFT [%] |
|---|---|---|---|
| 32 | 1024 | 160 | 15.6 |
| 128 | 16,384 | 896 | 5.46 |
| 1,024 | 1,048,576 | 10,240 | 0.97 |


## FFT en Python

Existen múltiples implementaciones de la FFT, sin embargo en este curso utilizaremos principalmente el módulo `fft` de la librería científica [`scipy` ](https://docs.scipy.org/doc/scipy/reference/tutorial/fft.html)

Si se require mayor eficiencia se puede considerar la librería [`pyFFTW` ](https://hgomersall.github.io/pyFFTW/) que es un wrapper de la *Fast Fourier Transform in the West* (FFTW), una famosa implementación escrita en lenguaje C

La principal función que utilizaremos es:

```python
scipy.fft.fft(x, # Un arreglo de NumPy,
              n=None, # El número de muestras que se usarán para calcular la FFT
              axis=-1, # El eje del arreglo en que se calculara la FFT,
              workers=None, # Número de trabajadores para calcular la FFT en paralelo
              ...)
```

que calcula la FFT de un arreglo de NumPy. Por convención, esta función retorna un arreglo con el espectro de frecuencia ordenando según 

$$
f = \left[0, \frac{F_s}{N},  \frac{2 F_s}{N}, \ldots, \frac{F_s}{2},  \ldots  -\frac{2 F_s}{N},   -\frac{F_s}{N} \right]
$$ 

es decir que primero retorna las frecuencias positivas y luego las negativas en orden invertido

Podemos usar la función:

```python
scipy.fft.fftshift(x, #  Un arreglo de numpy
                  ...
                  )
```

para dar vuelta la primera y segunda mitad del espectro obteniendo un orden más natural

También podemos usar:

```python
scipy.fft.fftfreq(n, #  El número de muestras de x
                  d=1.0 # El inverso de la frecuencia de muestreo
                 )
```

para calcular las frecuencias asociadas a la FFT (útil para graficar)



### Ejemplo

Sea una señal con dos componentes sinusoidales a frecuencia $1$ y $4$ Hz, respetivamente, y con desfase $0$ y $\pi/3 \approx 1.0472$, respectivamente

Calculemos los espectros de magnitud y fase usando la FFT

In [ ]:
import scipy.fft as sfft

Fs = 40
t = np.arange(0, 5, step=1./Fs); 
s = np.cos(2.0*np.pi*t) + 0.25*np.cos(2.0*np.pi*4*t + np.pi/3)

f = sfft.fftshift(sfft.fftfreq(n=len(t), d=1./Fs))
S = sfft.fftshift(sfft.fft(s))
SA = np.absolute(S) # Espectro de magnitud 
SP = np.angle(S) # Espectro fase

In [ ]:
SA_plot = hv.Curve((f, SA), 'Frecuencia', 'Espectro de magnitud').opts(width=350)
SP_plot = hv.Curve((f, SP), 'Frecuencia', 'Espectro de fase').opts(width=350)
SA_plot + SP_plot

Considerando sólo las frecuencias positivas (mitad derecha del espectro) podemos buscar la frecuencia de los dos puntos más altos del espectro de magnitud

In [ ]:
mask = np.argsort(SA[f>0])[-2:] # Ordena de menor a mayor y me quedo con las últimas dos
f[f>0][mask]

Las fases asociadas a estas frecuencias son

In [ ]:
SP[f>0][mask]

Es decir que hemos logrado recuperar las frecuencias y las fases de los componentes individuales

Comparemos ahora la diferencia en tiempo de cómputo entra la DFT calculada como producto matricial y la FFT ¿Cuántos ordenes de magnitud hay de diferencia?

In [ ]:
tf = t[:, np.newaxis]*f[:, np.newaxis].T
print("DFT numpy")
%timeit -n10 -r10 np.dot(s, np.exp(-1j*2.0*np.pi*tf));
print("FFT fftpack")
%timeit -n10 -r10 sfft.fftshift(sfft.fft(s))